Goal: Use a simple genetic algorithm (as in Holland’s and Goldberg’s “simple GA”) to approximate the unknown plaintext a such that
rsa_lib.encrypt_text(candidate, public_key, n) is as close as possible (in mean squared error) to the given ciphertext b.

In [1]:
import random
from pathlib import Path

# Import your RSA library
from var import rsa_lib

In [2]:
# Directly from 8_encrypted_text.txt (variant 8)
ciphertext = [
    4012, 216, 1403, 10437, 8192, 15617, 1403, 8192, 216, 17270, 696, 1403,
    8192, 12593, 216, 19224, 18807, 18807, 15951, 1403, 13821, 8192, 11101,
    18807, 18807, 8192, 20733, 216, 7555, 12593, 8192, 31197, 11101, 692,
    20733, 17270, 15951, 8192, 30299, 11101, 7555, 15951
]

public_key = 65537
n = 33227

len(ciphertext), public_key, n

(42, 65537, 33227)

In [3]:
ALPHABET = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ, "
ALPHABET_SIZE = len(ALPHABET)
TEXT_LENGTH = 42  # must match the length of the unknown plaintext

In [4]:
def random_char():
    return random.choice(ALPHABET)

def random_text(length=TEXT_LENGTH):
    return "".join(random_char() for _ in range(length))

In [5]:
def encrypt_candidate(text):
    """
    Encrypt candidate plaintext using the given rsa_lib.
    Returns a list (or iterable) of integers.
    """
    return rsa_lib.encrypt_text(text, public_key, n)

Define the error \( P \) and fitness:

$$
P(x, b) = \frac{1}{n} \sum_{i=1}^n (b'_i - b_i)^2
$$

and convert this to a fitness to maximize, e.g.:

$$
\text{F}(x) = \frac{1}{1 + P(x, b)}
$$

In [6]:
def error_P(candidate_text, target_cipher):
    cand_cipher = encrypt_candidate(candidate_text)

    # Safety: use min length if for some reason they differ
    L = min(len(cand_cipher), len(target_cipher))
    s = 0.0
    for i in range(L):
        diff = cand_cipher[i] - target_cipher[i]
        s += diff * diff
    return s / L

def fitness(candidate_text, target_cipher):
    P = error_P(candidate_text, target_cipher)
    return 1.0 / (1.0 + P)  # smaller error -> larger fitness

Initialize the population

In [7]:
POP_SIZE = 100

def initialize_population():
    return [random_text(TEXT_LENGTH) for _ in range(POP_SIZE)]

population = initialize_population()
len(population), population[0][:10]

(100, 'ABpElADmfj')

Selection operator (tournament selection)

In [8]:
def tournament_select(population, fitnesses, tournament_size=3):
    """
    Return one selected individual from population using tournament selection.
    """
    selected_idx = random.randrange(len(population))
    for _ in range(tournament_size - 1):
        i = random.randrange(len(population))
        if fitnesses[i] > fitnesses[selected_idx]:
            selected_idx = i
    return population[selected_idx]

Crossover operator (one-point crossover on strings)

In [9]:
def one_point_crossover(parent1, parent2):
    assert len(parent1) == len(parent2)
    L = len(parent1)
    if L < 2:
        return parent1, parent2
    cut = random.randint(1, L - 1)  # cut point between 1 and L-1
    child1 = parent1[:cut] + parent2[cut:]
    child2 = parent2[:cut] + parent1[cut:]
    return child1, child2

Mutation operator ( with small probability per position, change a character to a random other character from the alphabet)

In [10]:
MUTATION_RATE = 0.02  # probability of mutating each character

def mutate(text):
    text_list = list(text)
    for i in range(len(text_list)):
        if random.random() < MUTATION_RATE:
            text_list[i] = random_char()
    return "".join(text_list)

One GA generation (with elitism)



In [11]:
CROSSOVER_RATE = 0.8
ELITE_COUNT = 2  # number of best individuals preserved each generation

In [12]:
def make_next_generation(population, target_cipher):
    # 1. Evaluate current population
    fitnesses = [fitness(ind, target_cipher) for ind in population]

    # 2. Sort by fitness (descending)
    pop_fit = list(zip(population, fitnesses))
    pop_fit.sort(key=lambda x: x[1], reverse=True)

    # 3. Elites
    new_population = [ind for ind, fit in pop_fit[:ELITE_COUNT]]

    # 4. Generate the rest via selection, crossover, mutation
    while len(new_population) < POP_SIZE:
        # Parent selection
        parent1 = tournament_select(population, fitnesses)
        parent2 = tournament_select(population, fitnesses)

        # Crossover
        if random.random() < CROSSOVER_RATE:
            child1, child2 = one_point_crossover(parent1, parent2)
        else:
            child1, child2 = parent1, parent2

        # Mutation
        child1 = mutate(child1)
        child2 = mutate(child2)

        new_population.append(child1)
        if len(new_population) < POP_SIZE:
            new_population.append(child2)

    # Also return current best individual for logging
    best_ind, best_fit = pop_fit[0]
    return new_population, best_ind, best_fit

In [15]:
MAX_GENERATIONS = 20000  # you can adjust
TARGET_ERROR_THRESHOLD = 0.0  # stop if exact match is found

population = initialize_population()

for generation in range(MAX_GENERATIONS):
    # Evaluate and evolve
    population, best_ind, best_fit = make_next_generation(population, ciphertext)

    # Compute error explicitly for reporting
    best_error = error_P(best_ind, ciphertext)

    # REQUIRED OUTPUT: best current result for this iteration
    print(f"Generation {generation:4d} | best error P = {best_error:.6f} | best text = {repr(best_ind)}")

    # Stop if we hit perfect match (P = 0)
    if best_error <= TARGET_ERROR_THRESHOLD:
        print("Exact match found, stopping.")
        break

Generation    0 | best error P = 105282051.523810 | best text = 'lHOAxFpwilPRoizOViLulFrUtR,R  y,aVADGyqVtP'
Generation    1 | best error P = 100236704.761905 | best text = 'ayNQAiyXoUbBZEEXDwsDQZGCqeuRiBuwTp stMYrRb'
Generation    2 | best error P = 83871063.595238 | best text = 'EWxotJddiDRsfNLQBWoxquZMYbQPOAiSJPLXNpuPJl'
Generation    3 | best error P = 81607290.666667 | best text = 'EWxodJdyiDRsfNLQBiLulFrUtR,R  y,aVADGMYrRb'
Generation    4 | best error P = 67965887.095238 | best text = 'EWxodJpwilPRoiHpIwiDSIchlhMIRJRCJPLXNpuPJl'
Generation    5 | best error P = 62409164.000000 | best text = 'EFxodJdydsDqdTAbTHyRfFrUtR,R  y,aVADGMYrRb'
Generation    6 | best error P = 59910404.333333 | best text = 'EWxotJdd lPqdTJbTHyRfFrUtR,R  y,aEADGMJrRb'
Generation    7 | best error P = 51228942.642857 | best text = 'EWxotJdd lPqdTJbTHyRfFrUlR,R  y,aEAQtxOGJl'
Generation    8 | best error P = 49241427.142857 | best text = 'EWxodJpwilPRoiHpIwyRfguQAhxUDsZSUqNzulCPJD'
Generation    9 | best err

Now check the result by decrypting the ciphertext with the found plaintext.
Assuming we have factored n to find p and q, and computed d.

In [17]:
# factoring n to find p and q
p, q = 149, 223          # 149 * 223 = 33227
phi = (p - 1) * (q - 1)
# find d, the modular inverse of e mod phi
def egcd(a, b):
    if b == 0:
        return (1, 0, a)
    x, y, g = egcd(b, a % b)
    return (y, x - (a // b) * y, g)

x, y, g = egcd(public_key, phi)
d = x % phi              # private exponent

#  decrypting the ciphertext
plaintext_chars = [chr(pow(c, d, n)) for c in ciphertext]
plaintext = "".join(plaintext_chars)
print(plaintext)

When we have shuffled off this mortal coil
